In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Experimentation
# Change the depth map into whatever image you want (jpg, jpeg) and then run the python notebook
# It will generate 5 patterns 
depth_map = Image.open('input/dog.jpg').convert('L')  # Convert to grayscale
depth_array = np.array(depth_map)

plt.figure(figsize=(6, 6))  # Adjust the figure size
plt.imshow(depth_array, cmap='gray')  # Show as grayscale
plt.title('Depth Map')  # Add a title
plt.axis('off')  # Hide axes

In [ ]:
from PIL import Image
import os
import numpy as np
import cv2  # For adjust_color function

# Directory containing base patterns
patterns_dir = 'patterns/'

pattern_files = [f for f in os.listdir(patterns_dir) if f.endswith('.png')]

for pattern_file in pattern_files:
    # Load the base pattern
    base_pattern = Image.open(os.path.join(patterns_dir, pattern_file))
    base_pattern_array = np.array(base_pattern)

    # Get dimensions
    height, width = depth_array.shape
    base_height, base_width, _ = base_pattern_array.shape
    max_depth = np.max(depth_array)

    # Initialize the autostereogram
    autostereogram = np.zeros((height, width, 3), dtype=np.uint8)

    # Function to adjust color based on depth
    def adjust_color(pixel, depth_value, max_depth):
        normalized_depth = depth_value / max_depth
        adjusted_pixel = pixel * (1 - normalized_depth * 0.5)  # Reduce brightness by depth
        return np.clip(adjusted_pixel, 0, 255).astype(np.uint8)

    # Create the autostereogram
    for y in range(height):
        for x in range(width):
            depth_value = depth_array[y, x]

            # Fetch the corresponding pixel from the base pattern (tiled)
            pattern_x = x % base_width
            pattern_y = y % base_height
            pattern_pixel = base_pattern_array[pattern_y, pattern_x]

            # Adjust the color of the base pattern pixel based on depth
            autostereogram[y, x] = adjust_color(pattern_pixel, depth_value, max_depth)

    # Save the autostereogram with a unique filename
    output_filename = f'autostereogram_{os.path.splitext(pattern_file)[0]}.png'
    final_image = Image.fromarray(autostereogram)
    final_image.save(output_filename)


In [ ]:
# Experimenting with different color channels
import numpy as np
from PIL import Image
import os

# Directory to save generated autostereograms
output_dir = 'autostereograms'
os.makedirs(output_dir, exist_ok=True)

# Function to generate a random base pattern with a specific color theme
def generate_pattern(height, width, color_theme):
    pattern = np.zeros((height, width, 3), dtype=np.uint8)
    if color_theme == "blue":
        pattern[..., 2] = np.random.randint(0, 256, (height, width))  # Blue channel
    elif color_theme == "red":
        pattern[..., 0] = np.random.randint(0, 256, (height, width))  # Red channel
    elif color_theme == "green":
        pattern[..., 1] = np.random.randint(0, 256, (height, width))  # Green channel
    elif color_theme == "red_blue":
        pattern[..., 0] = np.random.randint(0, 256, (height, width))  # Red channel
        pattern[..., 2] = np.random.randint(0, 256, (height, width))  # Blue channel
    elif color_theme == "red_no_blue_green":
        pattern[..., 0] = np.random.randint(0, 256, (height, width))  # Red channel
    return pattern

# Color themes to randomize through
color_themes = ["blue", "red", "green", "red_blue", "red_no_blue_green"]

# Dimensions for the base pattern
pattern_height = 256
pattern_width = 256

# Get dimensions of the autostereogram
height, width = depth_array.shape
max_depth = np.max(depth_array)

# Generate autostereograms for each color theme
for color_theme in color_themes:
    # Generate the base pattern
    base_pattern_array = generate_pattern(pattern_height, pattern_width, color_theme)

    # Initialize the autostereogram
    autostereogram = np.zeros((height, width, 3), dtype=np.uint8)

    # Create the autostereogram
    for y in range(height):
        for x in range(width):
            depth_value = depth_array[y, x]

            # Fetch the corresponding pixel from the base pattern (tiled)
            pattern_x = x % pattern_width
            pattern_y = y % pattern_height
            pattern_pixel = base_pattern_array[pattern_y, pattern_x]

            # Adjust the color of the base pattern pixel based on depth
            autostereogram[y, x] = adjust_color(pattern_pixel, depth_value, max_depth)

    # Save the autostereogram with a unique filename
    output_filename = f'autostereogram_{color_theme}.png'
    final_image = Image.fromarray(autostereogram)
    final_image.save(os.path.join(output_dir, output_filename))

    # Optionally display the result
